# Exemplo de uso

## Parte 2: Aplicação do modelo de LDA.

Vamos seguir a partir pré-processamento realizado na etapa anterior.

Caso queira economizar tempo e não rodar os blocos de código da etapa 1, dado que ela pode demorar bastante tempo dependendo da máquina, pode-se importar o arquivo já processado disponibilizado `tweets_prefeitos_2020_preprocessado.csv.gzip` e seguir a partir daqui.

Importando as bilbiotecas que serão utilizadas.

In [9]:
import pandas as pd
import re
import gensim
import gensim.corpora as corpora
import pyLDAvis.gensim
import pyLDAvis
from pprint import pprint

Carregando o arquivo.

In [2]:
df_tweets = pd.read_csv('../dados/tweets_prefeitos_2020_preprocessado.csv.gzip', compression='gzip')

Criando a matriz de frequência de termos que será utilizada para treinar o modelo de LDA.

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True removes punctuations

data = df_tweets['text_processed'].values.tolist()
data_words = list(sent_to_words(data))

id2word = corpora.Dictionary(data_words)            # Create Dictionary
texts = data_words                                  # Create Corpus
corpus = [id2word.doc2bow(text) for text in texts]  # Term Document Frequency

Rodando o modelo de LDA para um total de 5 tópicos.

In [4]:
num_topics = 5
passes = 10

lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=id2word,
                                   num_topics=num_topics,
                                   passes=passes,
                                   random_state=1000)
doc_lda = lda_model[corpus]

pprint(lda_model.print_topics())

[(0,
  '0.042*"dose" + 0.033*"ser" + 0.032*"vacinacao" + 0.025*"vacinar" + '
  '0.019*"ano" + 0.018*"vacina" + 0.016*"estar" + 0.015*"pessoa" + 0.013*"dia" '
  '+ 0.013*"covid"'),
 (1,
  '0.024*"ser" + 0.017*"vacina" + 0.015*"vacinacao" + 0.012*"todo" + '
  '0.012*"covid" + 0.010*"contra" + 0.009*"saude" + 0.009*"pandemia" + '
  '0.008*"estar" + 0.006*"ter"'),
 (2,
  '0.045*"ser" + 0.020*"crianca" + 0.011*"mulher" + 0.009*"projeto" + '
  '0.008*"direito" + 0.008*"publico" + 0.007*"lei" + 0.007*"ano" + '
  '0.007*"contra" + 0.006*"sobre"'),
 (3,
  '0.057*"ser" + 0.023*"vacina" + 0.022*"bolsonaro" + 0.019*"ter" + '
  '0.012*"governo" + 0.010*"estar" + 0.010*"brasil" + 0.008*"fazer" + '
  '0.008*"poder" + 0.007*"morte"'),
 (4,
  '0.020*"ser" + 0.013*"dia" + 0.012*"todo" + 0.011*"mae" + 0.010*"pai" + '
  '0.010*"hoje" + 0.009*"filho" + 0.009*"amigo" + 0.008*"ano" + '
  '0.007*"familia"')]


Calculando a distribuição de tópicos por documento.

In [5]:
topic_dist = [lda_model.get_document_topics(item, minimum_probability=0.0) for item in corpus]
df_tweets.reset_index(drop=True, inplace=True)

for n in range(0, num_topics):
    for l in range(0, len(topic_dist)):
        df_tweets.at[l, f'topic_{n}_prob'] = topic_dist[l][n][1]

Exibindo os 20 *tweets* mais prováveis de cada um dos tópicos.

In [6]:
for n in range(0, num_topics):
    print('==========================================================================================================')
    print(f'Tópico {n+1}:')
    print('==========================================================================================================')
    for t in df_tweets.sort_values(f'topic_{n}_prob', ascending=False)[:20]['text']:
        print(t, end='\n\n')

Tópico 1:
O mutirão de vacinação contra covid-19 continua neste domingo. Veja quem pode se vacinar \n🔺1° dose: pessoas com 12 anos ou +\n🔺1° ou 2° dose: pessoas com 18 anos ou +\n🔺3° dose: profissionais da saúde com 2° dose até julho\n🔺3° dose: idosos com 70 anos ou + com 2° dose até julho

Para ser atendido na UBS, é necessário apenas apresentar o cartão do SUS. Caso o idoso não possua o cartão, basta levar originais e cópias de um documento de identificação com foto, CPF e comprovante de residência. O cadastro é feito na hora e a vacinação poderá ser realizada.

@sheric_ @prefeiturabelem ASesma informa que nesta terça-feira, 6, não haverá imunização contra a covid-19 em Belém. O município aguarda a chegada de novas doses. Caso novos imunizantes cheguem até esta terça-terça, nova etapa de vacinação será anunciada pela Sesma para ser retomada na quarta-feira, 7.

Domingo também é dia de vacina! \n\nOs 6 pontos fixos estarão funcionado normalmente hoje de 9h às 16h. Além dos drive-thrus

Visualizando o resultado com o auxílio da biblioteca `pyLDAvis`.

In [7]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, doc_lda, id2word, sort_topics=False)   
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.132170 -0.167923       1        1  10.074627
1     -0.111709  0.027078       2        1  23.474015
2      0.108430  0.223426       3        1  15.821437
3     -0.125114  0.056726       4        1  10.209609
4      0.260563 -0.139308       5        1  40.420312, topic_info=          Term        Freq       Total Category  logprob  loglift
41         ser  750.000000  750.000000  Default  30.0000  30.0000
613       dose  104.000000  104.000000  Default  29.0000  29.0000
257  vacinacao  171.000000  171.000000  Default  28.0000  28.0000
209     vacina  204.000000  204.000000  Default  27.0000  27.0000
522    crianca  147.000000  147.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
56       estar   61.678043  197.225301   Topic5  -5.0992  -0.2566
293        ter   56.376162  190.304613   Topic5  -5.1891  -0.3107
302      fazer   43.026231   97.849027   Topic5  -5.4594   0.0842
28          ir   37.947299  119.397452   Topic5  -5.5850  -0.2404
449       vida   32.199453   89.761479   Topic5  -5.7492  -0.1194

[355 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
5628      5  0.975085    abertura
6568      3  0.937700      aborto
2309      3  1.075140  absorvente
891       4  0.958508       achar
147       1  0.888174       acima
...     ...       ...         ...
449       3  0.111406        vida
449       4  0.178250        vida
449       5  0.356500        vida
466       3  1.025496   violencia
4004      3  1.007440       votar

[478 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

Criando uma coluna chamada `td` contendo a distribuição de tópicos para cada documento (*topic distribution*).

In [10]:
topics_cols = [x for x in df_tweets.columns.to_list() if re.match(r'topic_\d+_prob', x)]

df_tweets['topic_num'] = df_tweets[topics_cols].idxmax(axis=1)
df_tweets['topic_num'] = df_tweets['topic_num'].apply(lambda x: int(re.findall(r'(\d+)', x)[0]))

df_tweets['td'] = df_tweets[topics_cols].apply(lambda row: [x for x in row], axis=1)

Agora, quem estiver analisando os dados, atráves da visualização forncedida pela ferramenta `pyLDAvis`, assim como analisando as postagens mais prováveis de cada tópico, pode tentar atribuir um nome significativo para os tópicos encontrados pelo modelo de LDA.

In [11]:
df_tweets['topic_label'] = df_tweets['topic_num'].map({
    0: 'Tópico 1',
    1: 'Tópico 2',
    2: 'Tópico 3',
    3: 'Tópico 4',
    4: 'Tópico 5'
})

Salvando o dataset já contendo as distribuições de tópico.

In [12]:
df_tweets.to_csv('../dados/tweets_prefeitos_2020_preprocessado_com_td.csv.gzip', compression='gzip', index=False)